In [1]:
import os, argparse, glob
import pandas as pd
import numpy as np
import torch
from collections import Counter
import matplotlib.pyplot as plt
import joblib
from sklearn.preprocessing import LabelEncoder
from functools import partial
# from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
# from datasets import Dataset
# from peft import PeftModel, prepare_model_for_kbit_training

from IPython.display import display, Latex # Used for displaying formatted text in notebooks
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from tqdm import tqdm
import optuna

import shutil
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

In [2]:
# FOLD_IDX = 3
# df = pd.read_csv(f"./qwen3_eval_fold{FOLD_IDX}.csv")
# df["Misconception"] = df["Misconception"].fillna("NA")
# df["misconception_target"] = df["Category"].apply(lambda x: x.split("_")[-1]+":") + df["Misconception"]
# df["split_key"] = (df['QuestionId'].astype(str) + "_" + df['misconception_target'].astype(str)).astype('category').cat.codes

# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# fold_indices = list(skf.split(df, df['split_key']))
# tr_idx, va_idx = fold_indices[FOLD_IDX]
# df_tr, df_va = df.iloc[tr_idx].copy(), df.iloc[va_idx].copy()

In [3]:
# Reconstruct candidate-level data
def make_ranker_data(df, le):
    rows = []
    for idx, row in df.iterrows():
        qid = idx   # each explanation is its own query
        gt = row["misconception_target"]
        for class_name in le.classes_:
            rows.append({
                "qid": qid,
                "QuestionId": row["QuestionId"],
                "is_correct": row["is_correct"],
                "candidate": class_name,
                "rank_feature": row[f"{class_name}_rank"],
                "prob_feature": row[f"{class_name}_prob"],
                "label": int(class_name == gt),
            })
    return pd.DataFrame(rows)

def prepare_lgb_data(df):
    # load label_encoder.joblib
    le = joblib.load("./label_encoder.joblib")

    df_bgm = df.copy()
    df_bgm = df_bgm[["QuestionId", "is_correct", "misconception_target", "Pred_Targets", "Corresponding Probs", "split_key"]]
    df_bgm["Pred_Targets_list"] = df_bgm["Pred_Targets"].str.split("|")
    df_bgm["Pred_Probs_list"] = df_bgm["Corresponding Probs"].str.split("|").apply(lambda x: [float(p) for p in x])
    for class_name in le.classes_:
        df_bgm[f"{class_name}_rank"] = df_bgm.apply(lambda x: x["Pred_Targets_list"].index(class_name), axis=1)
        df_bgm[f"{class_name}_prob"] = df_bgm.apply(lambda x: x["Pred_Probs_list"][x[f"{class_name}_rank"]], axis=1)

    return df_bgm, le



In [4]:
def predict_topk(df_bgm, ranker, le, k=3):
    # Build candidate table in batch (no labels needed)
    rows = []
    for idx, row in df_bgm.iterrows():
        for class_name in le.classes_:
            rows.append({
                "qid": idx,
                "QuestionId": row["QuestionId"],
                "is_correct": row["is_correct"],
                "candidate": class_name,
                "rank_feature": row[f"{class_name}_rank"],
                "prob_feature": row[f"{class_name}_prob"],
            })
    cand_df = pd.DataFrame(rows)

    # Cast categorical if needed
    cand_df["QuestionId"] = cand_df["QuestionId"].astype("category")
    cand_df["candidate"] = cand_df["candidate"].astype("category")

    # Predict in one shot
    features = ["QuestionId", "candidate", "is_correct", "rank_feature", "prob_feature"]
    # features = ["QuestionId", "candidate", "is_correct", "prob_feature"]
    cand_df["score"] = ranker.predict(cand_df[features])

    # For each query, sort by score and take top-k
    topk_df = (
        cand_df.sort_values(["qid", "score"], ascending=[True, False])
               .groupby("qid")["candidate"].apply(lambda s: list(s.head(k)))
    )

    # Map back to original order
    return topk_df.reindex(df_bgm.index).to_dict()

def mapk(actual, predicted, k=3):
    score = 0.0
    for a, p in zip(actual, predicted):
        try:
            idx = p[:k].index(a)
            score += 1.0 / (idx + 1)
        except ValueError:
            continue
    return score / len(actual)

In [5]:
def objective_cv(trial, train_df, valid_df, le):
    # --- Suggest hyperparameters ---
    params = {
        "objective": "lambdarank",
        "metric": "map",
        "eval_at": [3],
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 0.1),
        "num_leaves": trial.suggest_int("num_leaves", 3, 60),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.9, 1.0),
        # "bagging_fraction": trial.suggest_uniform("bagging_fraction", 1.0),
        # "bagging_freq": 1,
        # "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-3, 10.0),
        # "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-3, 10.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
        "max_bin": trial.suggest_int("max_bin", 10, 255),
        "verbose": -1,
        "num_boost_round": trial.suggest_int("num_boost_round", 50, 1000),
    }


    train_df_actual = train_df.copy()
    valid_df_actual = valid_df.copy()

    avg_map = 0.0
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    for fold, (train_idx, valid_idx) in enumerate(skf.split(valid_df_actual, valid_df_actual["split_key"])):
        train_df = valid_df_actual.iloc[train_idx]
        valid_df = valid_df_actual.iloc[valid_idx]

        # Candidate-level training data
        df_train_rank = make_ranker_data(train_df, le)
        features = ["QuestionId", "candidate", "is_correct", "rank_feature", "prob_feature"]
        X_train = df_train_rank[features]
        X_train["QuestionId"] = X_train["QuestionId"].astype("category")
        X_train["candidate"] = X_train["candidate"].astype("category")
        y_train = df_train_rank["label"]
        group_train = df_train_rank.groupby("qid").size().to_list()
        train_set = lgb.Dataset(X_train, label=y_train, group=group_train)
    
        df_valid_rank = make_ranker_data(valid_df, le)
        X_valid = df_valid_rank[features]
        X_valid["QuestionId"] = X_valid["QuestionId"].astype("category")
        X_valid["candidate"] = X_valid["candidate"].astype("category")
        y_valid = df_valid_rank["label"]
        group_valid = df_valid_rank.groupby("qid").size().to_list()
        valid_set = lgb.Dataset(X_valid, label=y_valid, group=group_valid)
    
        # Train ranker
        ranker = lgb.train(
            params,
            train_set,
            # num_boost_round=200,
            valid_sets=[valid_set],
            valid_names=["valid"],
            callbacks=[lgb.early_stopping(stopping_rounds=30, verbose=False)]
        )
    
        # Evaluate MAP@3 on validation
        pred_top3_valid = predict_topk(valid_df, ranker, le, k=3)
        actual_valid = valid_df["misconception_target"].tolist()
        predicted_valid = [pred_top3_valid[idx] for idx in valid_df.index]
        fold_map = mapk(actual_valid, predicted_valid, k=3)

        avg_map += fold_map / 5.0

    
    # Evaluate MAP@3 on training
    pred_top3_train = predict_topk(train_df_actual, ranker, le, k=3)
    actual_train = train_df_actual["misconception_target"].tolist()
    predicted_train = [pred_top3_train[idx] for idx in train_df_actual.index]
    train_map = mapk(actual_train, predicted_train, k=3)

    # Log train MAP for later inspection
    trial.set_user_attr("train_map", train_map)
    
    # Print both for real-time feedback
    print(f"[Trial {trial.number}] Train MAP@3: {train_map:.4f}, Valid 5-CV AVG MAP@3: {avg_map:.4f}")
    
    return avg_map

In [6]:
def objective(trial, train_df, valid_df, le):
    # --- Suggest hyperparameters ---
    params = {
        "objective": "lambdarank",
        "metric": "map",
        "eval_at": [3],
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-3, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 2, 30),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.5, 1.0),
        # "bagging_fraction": trial.suggest_uniform("bagging_fraction", 1.0),
        # "bagging_freq": 1,
        # "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-3, 10.0),
        # "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-3, 10.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
        "max_bin": trial.suggest_int("max_bin", 10, 255),
        "verbose": -1,
        "num_boost_round": trial.suggest_int("num_boost_round", 50, 1000),
    }

    # Candidate-level training data
    df_train_rank = make_ranker_data(train_df, le)
    features = ["QuestionId", "candidate", "is_correct", "rank_feature", "prob_feature"]
    # features = ["QuestionId", "candidate", "is_correct", "prob_feature"]
    X_train = df_train_rank[features]
    X_train["QuestionId"] = X_train["QuestionId"].astype("category")
    X_train["candidate"] = X_train["candidate"].astype("category")
    y_train = df_train_rank["label"]
    group_train = df_train_rank.groupby("qid").size().to_list()
    train_set = lgb.Dataset(X_train, label=y_train, group=group_train)

    df_valid_rank = make_ranker_data(valid_df, le)
    X_valid = df_valid_rank[features]
    X_valid["QuestionId"] = X_valid["QuestionId"].astype("category")
    X_valid["candidate"] = X_valid["candidate"].astype("category")
    y_valid = df_valid_rank["label"]
    group_valid = df_valid_rank.groupby("qid").size().to_list()
    valid_set = lgb.Dataset(X_valid, label=y_valid, group=group_valid)

    # Train ranker
    ranker = lgb.train(
        params,
        train_set,
        # num_boost_round=200,
        valid_sets=[valid_set],
        valid_names=["valid"],
        callbacks=[lgb.early_stopping(stopping_rounds=100, verbose=False)]
    )

    # Evaluate MAP@3 on validation
    pred_top3_valid = predict_topk(valid_df, ranker, le, k=3)
    actual_valid = valid_df["misconception_target"].tolist()
    predicted_valid = [pred_top3_valid[idx] for idx in valid_df.index]
    fold_map = mapk(actual_valid, predicted_valid, k=3)

    # Evaluate MAP@3 on training
    pred_top3_train = predict_topk(train_df, ranker, le, k=3)
    actual_train = train_df["misconception_target"].tolist()
    predicted_train = [pred_top3_train[idx] for idx in train_df.index]
    train_map = mapk(actual_train, predicted_train, k=3)

    # Log train MAP for later inspection
    trial.set_user_attr("train_map", train_map)

    # Print both for real-time feedback
    print(f"[Trial {trial.number}] Train MAP@3: {train_map:.4f}, Valid MAP@3: {fold_map:.4f}")

    return fold_map

In [7]:

for FOLD_IDX in range(5):
    df = pd.read_csv(f"./qwen3_eval_fold{FOLD_IDX}.csv")
    df["Misconception"] = df["Misconception"].fillna("NA")
    df["misconception_target"] = df["Category"].apply(lambda x: x.split("_")[-1]+":") + df["Misconception"]
    df["split_key"] = (df['QuestionId'].astype(str) + "_" + df['misconception_target'].astype(str)).astype('category').cat.codes
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    fold_indices = list(skf.split(df, df['split_key']))
    tr_idx, va_idx = fold_indices[FOLD_IDX]
    df_tr, df_va = df.iloc[tr_idx].copy(), df.iloc[va_idx].copy()
    
    # --- Run Optuna study ---
    # Wrap the objective with extra arguments
    
    train_df, le = prepare_lgb_data(df_tr)
    valid_df, _ = prepare_lgb_data(df_va)
    
    # --- Train original MAP@3 ---
    pred_train = train_df["Pred_Targets_list"].apply(lambda x: x[:3])   # take top-3
    actual_train = train_df["misconception_target"]
    map_train_original = mapk(actual_train, pred_train, k=3)
    
    # --- Valid original MAP@3 ---
    pred_valid = valid_df["Pred_Targets_list"].apply(lambda x: x[:3])
    actual_valid = valid_df["misconception_target"]
    map_valid_original = mapk(actual_valid, pred_valid, k=3)
    
    print(f"Original MAP@3 (train): {map_train_original:.4f}")
    print(f"Original MAP@3 (valid): {map_valid_original:.4f}")
    
    
    objective_fn = partial(objective, train_df=train_df, valid_df=valid_df, le=le)
    
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_fn, n_trials=50)   # increase n_trials for better search
    
    print("Best AVG MAP@3:", study.best_value)
    print("Best params:", study.best_params)
    
    
    # --- Train final model with best params on full data ---
    best_params = study.best_params
    best_params.update({
        "objective": "lambdarank",
        "metric": "map",
        "eval_at": [3],
        "verbose": -1
    })
    
    df_rank_full = make_ranker_data(train_df, le)
    X_full = df_rank_full[["QuestionId", "candidate", "is_correct", "rank_feature", "prob_feature"]]
    X_full["QuestionId"] = X_full["QuestionId"].astype("category")
    X_full["candidate"] = X_full["candidate"].astype("category")
    y_full = df_rank_full["label"]
    group_full = df_rank_full.groupby("qid").size().to_list()
    
    train_set_full = lgb.Dataset(X_full, label=y_full, group=group_full)
    final_model = lgb.train(best_params, train_set_full, num_boost_round=300)
    
    print("Final model trained with best hyperparameters.")
    
    joblib.dump(final_model, f"lgbm_ranker_f{FOLD_IDX}.pkl")

# # --- Train final model with best params on full data ---
# best_params = study.best_params
# best_params.update({
#     "objective": "lambdarank",
#     "metric": "map",
#     "eval_at": [3],
#     "verbose": -1
# })

# # Candidate-level training data
# df_train_rank = make_ranker_data(train_df, le)
# df_valid_rank = make_ranker_data(valid_df, le)

# features = ["QuestionId", "candidate", "is_correct", "rank_feature", "prob_feature"]

# # --- Train set ---
# X_train = df_train_rank[features].copy()
# X_train["QuestionId"] = X_train["QuestionId"].astype("category")
# X_train["candidate"] = X_train["candidate"].astype("category")
# y_train = df_train_rank["label"]
# group_train = df_train_rank.groupby("qid").size().to_list()
# train_set = lgb.Dataset(X_train, label=y_train, group=group_train)

# # --- Valid set ---
# X_valid = df_valid_rank[features].copy()
# X_valid["QuestionId"] = X_valid["QuestionId"].astype("category")
# X_valid["candidate"] = X_valid["candidate"].astype("category")
# y_valid = df_valid_rank["label"]
# group_valid = df_valid_rank.groupby("qid").size().to_list()
# valid_set = lgb.Dataset(X_valid, label=y_valid, group=group_valid)

# # --- Train final model ---
# final_model = lgb.train(
#     best_params,
#     train_set,
#     # num_boost_round=200,
#     valid_sets=[valid_set],
#     valid_names=["valid"],
#     callbacks=[lgb.early_stopping(stopping_rounds=30, verbose=False)]
# )

# print("Final model trained on train_df and validated on valid_df.")

[I 2025-09-28 09:20:37,193] A new study created in memory with name: no-name-3aa70b29-ddb9-4546-96d9-94768f8eb07d


Original MAP@3 (train): 0.9799
Original MAP@3 (valid): 0.9476


[I 2025-09-28 09:21:08,159] Trial 0 finished with value: 0.9476158038147134 and parameters: {'learning_rate': 0.014060218470631623, 'num_leaves': 8, 'feature_fraction': 0.5515452186923009, 'min_data_in_leaf': 92, 'max_bin': 211, 'num_boost_round': 953}. Best is trial 0 with value: 0.9476158038147134.


[Trial 0] Train MAP@3: 0.9799, Valid MAP@3: 0.9476


[I 2025-09-28 09:21:40,640] Trial 1 finished with value: 0.9476385104450492 and parameters: {'learning_rate': 0.03820374849597256, 'num_leaves': 9, 'feature_fraction': 0.6961191865743366, 'min_data_in_leaf': 49, 'max_bin': 195, 'num_boost_round': 279}. Best is trial 1 with value: 0.9476385104450492.


...

[Trial 47] Train MAP@3: 0.9803, Valid MAP@3: 0.9493


[I 2025-09-28 11:36:48,746] Trial 48 finished with value: 0.9496071217695413 and parameters: {'learning_rate': 0.03663504803669234, 'num_leaves': 23, 'feature_fraction': 0.7972403450540073, 'min_data_in_leaf': 46, 'max_bin': 177, 'num_boost_round': 126}. Best is trial 43 with value: 0.9503792523958755.


[Trial 48] Train MAP@3: 0.9807, Valid MAP@3: 0.9496


[I 2025-09-28 11:37:21,285] Trial 49 finished with value: 0.9493118953535898 and parameters: {'learning_rate': 0.01573437311277939, 'num_leaves': 29, 'feature_fraction': 0.8821851049811714, 'min_data_in_leaf': 28, 'max_bin': 202, 'num_boost_round': 222}. Best is trial 43 with value: 0.9503792523958755.


[Trial 49] Train MAP@3: 0.9803, Valid MAP@3: 0.9493
Best AVG MAP@3: 0.9503792523958755
Best params: {'learning_rate': 0.0492070774100836, 'num_leaves': 22, 'feature_fraction': 0.7647984923331111, 'min_data_in_leaf': 64, 'max_bin': 190, 'num_boost_round': 243}
Final model trained with best hyperparameters.


In [8]:
# joblib.dump(final_model, f"lgbm_ranker_f{FOLD_IDX}.pkl")

['lgbm_ranker_f3.pkl']

In [9]:
assert False

AssertionError: 

In [ ]:
pred_top3 = predict_topk(df_bgm, final_model, le, k=3)

actual = df_bgm["misconception_target"].tolist()
predicted = [pred_top3[idx] for idx in df_bgm.index]
print("MAP@3:", mapk(actual, predicted, k=3))

In [ ]:
joblib.dump(final_model, "lgbm_ranker_v2.pkl")

In [ ]:
import optuna

# Add split_key
df_bgm["split_key"] = df_bgm["QuestionId"].astype(str) + "_" + df_bgm["misconception_target"].astype(str)

# If a split_key appears only once, duplicate it
vc = df_bgm["split_key"].value_counts()
rare_keys = vc[vc == 1].index
df_dup = df_bgm[df_bgm["split_key"].isin(rare_keys)].copy()
df_bgm = pd.concat([df_bgm, df_dup], ignore_index=True)

# Build folds
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
avg = 0.0
for fold, (train_idx, valid_idx) in enumerate(skf.split(df_bgm, df_bgm["split_key"])):
    print(f"\n===== Fold {fold} =====")

    train_df = df_bgm.iloc[train_idx]
    valid_df = df_bgm.iloc[valid_idx]

    # --- Candidate-level training data ---
    df_rank = make_ranker_data(train_df, le)
    features = ["QuestionId", "candidate", "is_correct", "rank_feature", "prob_feature"]
    X_train = df_rank[features]
    X_train["QuestionId"] = X_train["QuestionId"].astype("category")
    X_train["candidate"] = X_train["candidate"].astype("category")
    y_train = df_rank["label"]
    group_train = df_rank.groupby("qid").size().to_list()

    train_set = lgb.Dataset(X_train, label=y_train, group=group_train)

    # Validation set (candidate-level)
    df_valid_rank = make_ranker_data(valid_df, le)
    X_valid = df_valid_rank[features]
    X_valid["QuestionId"] = X_valid["QuestionId"].astype("category")
    X_valid["candidate"] = X_valid["candidate"].astype("category")
    y_valid = df_valid_rank["label"]
    group_valid = df_valid_rank.groupby("qid").size().to_list()

    valid_set = lgb.Dataset(X_valid, label=y_valid, group=group_valid)

    # --- Train ranker ---
    params = {
        "objective": "lambdarank",
        "metric": "map",
        "eval_at": [3],
        "learning_rate": 0.1,
        "feature_fraction": 1,
        "num_leaves": 11,
        "verbose": -1,
        "max_bin": 230,
        "min_data_in_leaf": 100,
    }

    ranker = lgb.train(
        params,
        train_set,
        num_boost_round=200,
        valid_sets=[valid_set],
        valid_names=["valid"],
        # verbose_eval=50
    )

    # --- Evaluate MAP@3 on validation ---
    pred_top3 = predict_topk(valid_df, ranker, le, k=3)
    actual = valid_df["misconception_target"].tolist()
    predicted = [pred_top3[idx] for idx in valid_df.index]
    mapk_score = mapk(actual, predicted, k=3)
    print("Fold MAP@3:", mapk_score)
    avg += mapk_score / 5.0

print("\n===== Average MAP@3:", avg)

In [ ]:
# Features and label
features = ["QuestionId", "candidate", "is_correct", "rank_feature", "prob_feature"]
X = df_rank[features]
X["QuestionId"] = X["QuestionId"].astype("category")
X["candidate"] = X["candidate"].astype("category")
y = df_rank["label"]

# Group info: how many candidates per query
group = df_rank.groupby("qid").size().to_list()

# LightGBM dataset
train_set = lgb.Dataset(X, label=y, group=group)

params = {
    "objective": "lambdarank",
    "metric": "map",
    "eval_at": [3],          # evaluate MAP@3
    "learning_rate": 0.05,
    "num_leaves": 30,
    # "min_data_in_leaf": 20,
    # "max_bin": 30,
    # "min_data_in_bin": 20,
    # "boosting": "gbdt",
    # "verbose": -1,
}

ranker = lgb.train(
    params,
    train_set,
    num_boost_round=200,
    valid_sets=[train_set],
    valid_names=["train"],
    # verbose_eval=20
)

pred_top3 = predict_topk(df_bgm, ranker, le, k=3)

actual = df_bgm["misconception_target"].tolist()
predicted = [pred_top3[idx] for idx in df_bgm.index]
print("MAP@3:", mapk(actual, predicted, k=3))


In [ ]:
joblib.dump(ranker, "lgbm_ranker.pkl")

loaded_ranker = joblib.load("lgbm_ranker.pkl")
pred_top3 = predict_topk(df_bgm, loaded_ranker, le, k=3)
predicted = [pred_top3[idx] for idx in df_bgm.index]
print("MAP@3:", mapk(actual, predicted, k=3))